In [1]:
using JuMP
using Gurobi
using Combinatorics
using ToeplitzMatrices
using LinearAlgebra

In [2]:
# Define parameters of the problem
A=[0.2 0; 0 1.3]
C=[0.5 1.5; 0 1]

n_y,n_x=size(C)

H_w=[1 0; -1 0; 0 1; 0 -1]
h_w=1.5*[1; 1; 1; 1]
n_w=size(h_w,1)


H_v=[1 0; -1 0; 0 1; 0 -1]
h_v=0.3*[1; 1; 1; 1]
n_v=size(h_v,1)

H_x0=[1 0; -1 0; 0 1; 0 -1]
h_x0=1.7*[1; 1; 1; 1]
n_x0=size(h_x0,1)

T=10
N=3


3

In [3]:
time_instants=0:T-1 
all_schedules=collect(combinations(time_instants,N)); # do we need to collect ?
n_schedules=size(all_schedules,1)

120

In [4]:
# compute the J matrix
J=I # Identity matrix
for i = 1:T
    J = [J; (A^i)] # not efficient
end

In [5]:
# compute the S matrix
S=zeros(n_x*(T+1),n_x*T)
l=one(A) # Is it efficient, command "I" is another option...
S[n_x+1:2*n_x,1:n_x]=l # A^0
for i=1:T-1 # loop over the lines
    l=[A*l[1:n_x,1:n_x] l] # A*l[1:nx,1:nx] is A^i
    S[(i+1)*n_x+1:(i+2)*n_x, 1:(i+1)*n_x]=l
end
print("We don't use the triangular structure...\n")

We don't use the triangular structure...


In [6]:
# function to compute C_M
print("We don't use the sparse structure...\n")
function computeC_M(T,M)
    # construct C_sched (=C_bar)
    M=M.+1 # to have indices
    diag=zeros(T)
    diag[M].=1
    dMat=Diagonal(diag)
    C_sched=kron(dMat,C)
    return C_sched
end

We don't use the sparse structure...


computeC_M (generic function with 1 method)

In [7]:
# compute H_eta and h_eta
H_eta=[kron(Matrix(1I,T,T), H_w) zeros(T*n_w,T*n_y + n_x) ;
    zeros(T*n_v, T*n_x) kron(Matrix(1I,T,T), H_v) zeros(T*n_v,n_x);
    zeros(n_x0,T*(n_x+n_y)) H_x0]

h_eta=[kron(ones(T),h_w); kron(ones(T),h_v); h_x0];

In [8]:
function synthesize_fhae(T,sched)
    # Design the optimal estimator
    model = Model(Gurobi.Optimizer)
    
    # Declare variables
    @variable(model,Q[1:n_x*T,1:n_x*T])
    @variable(model,r[1:n_x*T])
    @variable(model,Lambda[1:2*n_x*(T+1),1:T*(n_w+n_v)+n_x0])
    @variable(model,alpha0[1:T+1])
    
    alpha_bar=kron(alpha0,ones(n_x))
    alpha_bar=[alpha_bar; alpha_bar];
    
    # add constraints
    C_bar=zeros(T*n_y, (T+1)*n_x)
    
    C_bar[:,1:T*n_x] .= computeC_M(T,sched)
    
    P_xi_w = S + S*Q*C_bar*S
    
    P_xi_v = S*Q
    P_xi_xi0 = ( I + S*Q*C_bar )*J
    
    R_T_mT = [ I ; -Matrix(1I, n_x*(T+1), n_x*(T+1)) ]
    
    # Add dual equality constraint
    LHS1 = Lambda * H_eta
    RHS1 = R_T_mT * [ P_xi_w P_xi_v P_xi_xi0 ]
    @constraint(model, LHS1 .== RHS1 )
    
    # Add dual inequality constraint
    LHS2 = Lambda * h_eta    
    RHS2 = alpha_bar - R_T_mT * S*r
    @constraint(model, LHS2.<=RHS2 )
    
    # low diagonal constraint
    for blk_row_idx = 1:div(size(Q,1), n_x)-1
        LHS_bloc = Q[(blk_row_idx-1)*n_x.+(1:n_x),blk_row_idx*n_y+1:end]
        @constraint(model,LHS_bloc.==zeros(size(LHS_bloc)))
    end
    
    # positivity constraints
    @constraint(model,alpha0.>=0)
    @constraint(model,Lambda.>=zeros(size(Lambda)))
    
    # objective function
    @objective(model, Min, sum(alpha0))
    
    # optimize
    optimize!(model)
    
    # define outputs
    Q0 = value.(Q);
    r0 = value.(r);
    
    F = inv(I + Q0*C_bar*S ) * Q0; # slow
    
    
    f = inv(I + Q0*C_bar*S ) * r0; # slow
    f = reshape(f,2,:)
    
    cost=objective_value(model)
    
    return F, f, cost
end

synthesize_fhae (generic function with 1 method)

In [13]:
costs=zeros(n_schedules)

time_beginning=time_ns()

F=0
f=0

for (schedule_idx,temp_schedule) = enumerate(all_schedules)
    print("\n\nSchedule $(schedule_idx)/$(n_schedules): $(temp_schedule)\n")
    
    F,f,costs[schedule_idx] = synthesize_fhae(T,temp_schedule)
    print("sched=$(temp_schedule): f=$f\n")
end

time_ending=time_ns()
exec_time=(time_ending-time_beginning)/1e9
print("\n\n=========== exec_time = $(exec_time) seconds ===========\n\n")



Schedule 1/120: [0, 1, 2]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 19903 nonzeros
Model fingerprint: 0x0d37b650
Coefficient statistics:
  Matrix range     [1e-08, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
Presolve removed 5319 rows and 3137 columns
Presolve time: 0.01s
Presolved: 460 rows, 990 columns, 5920 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     223    1.2985949e+02   0.000000e+00   0.000000e+00      0s

Solved in 223 iterations and 0.01 seconds
Optimal objective  1.298594889e+02




Schedule 9/120: [0, 2, 3]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 21223 nonzeros
Model fingerprint: 0x176f8d20
Coefficient statistics:
  Matrix range     [2e-09, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
Presolve removed 5287 rows and 3081 columns
Presolve time: 0.01s
Presolved: 492 rows, 1046 columns, 6176 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     239    9.5582761e+01   0.000000e+00   0.000000e+00      0s

Solved in 239 iterations and 0.02 seconds
Optimal objective  9.558276071e+01



Schedule 17/120: [0, 3, 5]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 23203 nonzeros
Model fingerprint: 0x280b217b
Coefficient statistics:
  Matrix range     [8e-11, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
Presolve removed 5247 rows and 3013 columns
Presolve time: 0.01s
Presolved: 532 rows, 1114 columns, 5952 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     218    5.7345059e+01   0.000000e+00   0.000000e+00      0s

Solved in 218 iterations and 0.02 seconds
Optimal objective  5.734505900e+0



Schedule 23/120: [0, 4, 6]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 24523 nonzeros
Model fingerprint: 0x618d1a9f
Coefficient statistics:
  Matrix range     [2e-11, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
Presolve removed 5255 rows and 3037 columns
Presolve time: 0.01s
Presolved: 524 rows, 1090 columns, 5500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     210    5.2803860e+01   0.000000e+00   0.000000e+00      0s

Solved in 210 iterations and 0.01 seconds
Optimal objective  5.280386000e+0



Schedule 29/120: [0, 5, 8]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 26503 nonzeros
Model fingerprint: 0x4e69a3f5
Coefficient statistics:
  Matrix range     [7e-13, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5287 rows and 3113 columns
Presolve time: 0.01s
Presolved: 492 rows, 1014 columns, 4572 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     189    5.7345059e+01   0.00

...



Schedule 103/120: [4, 5, 8]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 29143 nonzeros
Model fingerprint: 0xaec83f80
Coefficient statistics:
  Matrix range     [7e-13, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5303 rows and 3145 columns
Presolve time: 0.01s
Presolved: 476 rows, 982 columns, 4908 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     186    5.5816870e+01   0.00



Schedule 109/120: [4, 7, 9]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 31123 nonzeros
Model fingerprint: 0x6f5f7963
Coefficient statistics:
  Matrix range     [1e-13, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 5327 rows and 3205 columns
Presolve time: 0.01s
Presolved: 452 rows, 922 columns, 4016 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 30
 Free vars  : 56
 AA'



Schedule 113/120: [5, 6, 9]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 31123 nonzeros
Model fingerprint: 0xb2cbfd65
Coefficient statistics:
  Matrix range     [1e-13, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 5361 rows and 3270 columns
Presolve time: 0.01s
Presolved: 418 rows, 857 columns, 3954 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 26
 Free vars  : 56
 AA'



Schedule 117/120: [6, 7, 8]

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5779 rows, 4127 columns and 31783 nonzeros
Model fingerprint: 0xe585a590
Coefficient statistics:
  Matrix range     [7e-13, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 5403 rows and 3355 columns
Presolve time: 0.01s
Presolved: 376 rows, 772 columns, 3436 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 30
 Free vars  : 52
 AA'



=========== exec_time = 9.792956784 seconds ===========



In [10]:
opt_idx=argmin(costs)
opt_cost=costs[opt_idx]
opt_schedule=all_schedules[opt_idx]

print("\nopt_schedule=$(opt_schedule), opt_cost=$(opt_cost)")


opt_schedule=[2, 5, 7], opt_cost=42.56219999999999

In [11]:
time_beginning=time_ns()

0x0000a43add78bc5b